In [1]:
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import bs4

import re
import time

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import pymysql

# 找到所有全員的id存入testdb中的player_info表格

In [73]:
#將所有球員的id抓起來放在database
all_player_info = []
count = 1
path   = r'/Users/andyhsu/Desktop/NBA/chromedriver'
driver = webdriver.Chrome(path)
url = 'https://www.nba.com/players'
driver.get(url)
time.sleep(5)
選單 = Select(driver.find_element_by_xpath('//select[@class="DropDown_select__5Rjt0" and @title="Page Number Selection Drown Down List"]'))
選單.select_by_value("-1")
大表 = driver.find_element_by_xpath('//table[@class="players-list"]')
球員列表 = 大表.find_elements_by_xpath('./tbody/tr')
for 球員資料 in 球員列表:
    球員_info = []
    href        = 球員資料.find_element_by_xpath('./td/a').get_attribute('href')
    player_id   = re.search('\d+',href)
    球員姓名 = 球員資料.find_element_by_xpath('./td/a/div[@class="flex flex-col lg:flex-row"]/p[@class="t6 mr-1"]').text +' '+\
            球員資料.find_element_by_xpath('./td/a/div[@class="flex flex-col lg:flex-row"]/p[@class="t6"]').text
    球員_info.append(球員姓名)
    球員_info.append(player_id[0])
    all_player_info.append(球員_info)
    count+=1
driver.close()


In [79]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS PLAYER_INFO")
sql = """CREATE TABLE PLAYER_INFO (
         NAME  CHAR(50) NOT NULL,
         ID CHAR(30))"""
cursor.execute(sql)
print("CREATE TABLE OK")
cursor.close()
db.close()

CREATE TABLE OK


In [81]:
id_records = [tuple (x) for x in all_player_info]
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()

sql = """INSERT INTO PLAYER_INFO (NAME, ID) VALUES (%s, %s)"""
val = id_records

mycursor.executemany(sql, val)

#提交任務
db.commit()
print(mycursor.rowcount, "was inserted.")

mycursor.close()
db.close()

618 was inserted.


# select from in 

In [35]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()

var = ['Grayson Allen','Jarrett Allen',]

#map把list內都變成str再轉成tuple後續用repr成為解釋器讀得懂的東西
sql = f"""SELECT ID FROM PLAYER_INFO WHERE NAME IN {repr(tuple(map(str, var)))}"""

print(sql)

mycursor.execute(sql)

mycursor.fetchall()

SELECT ID FROM PLAYER_INFO WHERE NAME IN ('Grayson Allen', 'Jarrett Allen')


(('1628960',), ('1628386',))

In [83]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()

sql = """SELECT ID FROM PLAYER_INFO WHERE NAME = 'Steven Adams'"""

mycursor.execute(sql)

#提交任務
print(mycursor.fetchone()[0])
print(mycursor.rowcount)

mycursor.close()
db.close()

203500
1


# 抓球員近50場的資料，並存入FOCUS_PLAYER的表格

In [57]:
path   = r'/Users/andyhsu/Desktop/NBA/chromedriver'
class NBA:
    def __init__(self,player):
        self.player = player

    def get_player_id(self,player_name):
        driver = webdriver.Chrome(path)
        url = 'https://www.nba.com/players'
        driver.get(url)
        time.sleep(5)
        選單 = Select(driver.find_element_by_xpath('//select[@class="DropDown_select__5Rjt0" and @title="Page Number Selection Drown Down List"]'))
        選單.select_by_value("-1")
        大表 = driver.find_element_by_xpath('//table[@class="players-list"]')
        球員列表 = 大表.find_elements_by_xpath('./tbody/tr')
        for 球員資料 in 球員列表:
            href        = 球員資料.find_element_by_xpath('./td/a').get_attribute('href')
            player_id   = re.search('\d+',href)
            球員姓名 = 球員資料.find_element_by_xpath('./td/a/div[@class="flex flex-col lg:flex-row"]/p[@class="t6 mr-1"]').text +' '+\
                    球員資料.find_element_by_xpath('./td/a/div[@class="flex flex-col lg:flex-row"]/p[@class="t6"]').text
            if 球員姓名 == player_name:
                break
        driver.close()
        return player_id[0]

    def get_current_data(self,player_id):
        driver = webdriver.Chrome(path)
        url         = 'https://www.nba.com/stats/player/'+ str(player_id)+ '/boxscores-traditional'


        url_text    = requests.get(url).text

        url_parse   = bs4.BeautifulSoup(url_text,"lxml")

        player_info = url_parse.find('div',class_='flex flex-col text-white')

        name        = []
        for p in player_info.find_all('p',class_='PlayerSummary_playerNameText__K7ZXO'):
            name.append(p.text)

        print(name)
        player_name = (' ').join(name)

        driver.get(url)
        time.sleep(3)

        標題資料    = []
        標題行   = driver.find_elements_by_xpath('//tr[@class = "Crom_headers__QJmqP"]/th')
        for 標題 in 標題行:
            標題資料.append(標題.text)




        表格    = driver.find_element_by_xpath('//tbody[@class = "Crom_body__2rqCI"]')




        每行資料 = 表格.find_elements_by_xpath('./tr')


        all_data = []

        for row in 每行資料:
            row_data      = []
            every_td_data = row.find_elements_by_xpath('./td')
            for index,data in enumerate(every_td_data):
                if index == 0:
                    correct_data = data.find_element_by_xpath('./a').text

                else:
                    correct_data = data.text
                row_data.append(correct_data)
            all_data.append(row_data)

        driver.close()
        return pd.DataFrame(all_data,columns = 標題資料)

    def current_50_avg(self,df):
        cur_50 = []
        cur_50.append(self.player)
        cur_50.append(round(df['MIN'].astype(int).mean(),2))
        cur_50.append(round(df['PTS'].astype(int).mean(),2))
        cur_50.append(round(df['REB'].astype(int).mean(),2))
        cur_50.append(round(df['AST'].astype(int).mean(),2))
        cur_50.append(round(df['STL'].astype(int).mean(),2))
        cur_50.append(round(df['BLK'].astype(int).mean(),2))
        cur_50.append(round(df['TOV'].astype(int).mean(),2))
        return cur_50
    

In [64]:
players = ['LaMelo Ball', 'Giannis Antetokounmpo', 'Lonzo Ball', 'Nikola Jokic']
all_data = []
for player in players:
    sd = NBA(player)
    player_id = sd.get_player_id(sd.player)
    print(player_id)
    data      = sd.get_current_data(player_id)
    current_data = sd.current_50_avg(data)
    all_data.append(current_data)

1630163
['LaMelo', 'Ball']
203507
['Giannis', 'Antetokounmpo']
1628366
['Lonzo', 'Ball']
203999
['Nikola', 'Jokic']


In [65]:
all_data

[['LaMelo Ball', 31.88, 20.06, 6.12, 7.28, 1.44, 0.4, 3.18],
 ['Giannis Antetokounmpo', 33.04, 30.9, 11.5, 5.72, 1.04, 1.18, 3.3],
 ['Lonzo Ball',
  34.65714285714286,
  13.0,
  5.428571428571429,
  5.085714285714285,
  1.8285714285714285,
  0.8857142857142857,
  2.342857142857143],
 ['Nikola Jokic', 33.82, 27.46, 13.92, 8.1, 1.54, 0.92, 3.98]]

In [66]:
# 開啟資料庫連線
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
# 使用 cursor() 方法建立一個遊標物件 cursor
cursor = db.cursor()
# 使用 execute() 方法執行 SQL，如果表存在則刪除
cursor.execute("DROP TABLE IF EXISTS FOCUS_PLAYER")
# 使用預處理語句建立表
sql = """CREATE TABLE FOCUS_PLAYER (
         NAME  CHAR(30) NOT NULL,
         MIN FLOAT(5,3),
         POINT FLOAT(4,2),  
         REB FLOAT(4,2),
         ASS FLOAT(4,2),
         STL FLOAT(4,2),
         BLK FLOAT(4,2),
         TOV FLOAT(4,2) )"""
cursor.execute(sql)
print("CREATE TABLE OK")
# 關閉資料庫連線
db.close()
# 修改表內資料才需要commit


CREATE TABLE OK


In [67]:
records = []
for x in all_data:
    print(tuple(x))
    records.append(tuple(x))

('LaMelo Ball', 31.88, 20.06, 6.12, 7.28, 1.44, 0.4, 3.18)
('Giannis Antetokounmpo', 33.04, 30.9, 11.5, 5.72, 1.04, 1.18, 3.3)
('Lonzo Ball', 34.65714285714286, 13.0, 5.428571428571429, 5.085714285714285, 1.8285714285714285, 0.8857142857142857, 2.342857142857143)
('Nikola Jokic', 33.82, 27.46, 13.92, 8.1, 1.54, 0.92, 3.98)


In [68]:
db = pymysql.connect(host = "localhost", user = "root", password = "airflow", db = "testdb" )
#要新增或修改資料庫內容要呼叫的指標物件
mycursor = db.cursor()

sql = """INSERT INTO FOCUS_PLAYER (NAME, MIN, POINT, REB, ASS, STL, BLK, TOV) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
val = records

mycursor.executemany(sql, val)

#提交任務
db.commit()
print(mycursor.rowcount, "was inserted.")

mycursor.close()
db.close()

4 was inserted.
